<a href="https://colab.research.google.com/github/alicetw40342/AI-Recommender-System/blob/main/Project2_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving movie_plots.csv to movie_plots.csv


In [6]:
!pip install scikit-learn
!pip install pandas

In [7]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 把下面改成你上傳的檔名
csv_file = "movie_plots.csv"

# 讀 CSV
df = pd.read_csv(f"/content/{csv_file}")

print("總共有幾筆：", df.shape[0])
print(df.head())

# 清理文字
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)     # 移除標點符號
    words = text.split()
    stopwords = set(TfidfVectorizer(stop_words='english').get_stop_words())
    words = [w for w in words if w not in stopwords]
    return ' '.join(words)

df['Cleaned_Plot'] = df['Plot'].apply(clean_text)

# 計算 TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['Cleaned_Plot'])

print("TF-IDF 矩陣大小：", tfidf_matrix.shape)

# 建立推薦函式
def recommend_movies(movie_title, top_n=5):
    if movie_title not in df['Title'].values:
        print(f"找不到 {movie_title}")
        return

    idx = df[df['Title'] == movie_title].index[0]

    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    similar_indices = cosine_sim.argsort()[-(top_n+1):-1][::-1]

    print(f"與《{movie_title}》劇情相似的 {top_n} 部電影：\n")
    for i in similar_indices:
        print(f"{df.iloc[i]['Title']} (Similarity: {cosine_sim[i]:.4f})")

# 測試推薦
recommend_movies("The Matrix")

總共有幾筆： 10
            Title                                               Plot
0      The Matrix  A computer hacker learns about the true nature...
1         Titanic  A young aristocrat falls in love with a poor a...
2       Inception  A thief who enters the dreams of others to ste...
3    Interstellar  A group of explorers travel through a wormhole...
4  The Terminator  A cyborg assassin is sent back in time to kill...
TF-IDF 矩陣大小： (10, 89)
與《The Matrix》劇情相似的 5 部電影：

Star Wars (Similarity: 0.0856)
Avatar (Similarity: 0.0000)
Gladiator (Similarity: 0.0000)
The Lord of the Rings (Similarity: 0.0000)
Jurassic Park (Similarity: 0.0000)
